<a href="https://colab.research.google.com/github/Larr014/DataMining/blob/master/Twitter_get_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tweepy                   # Python wrapper around Twitter API
from google.colab import drive  # to mount Drive to Colab notebook
import json
import csv
from datetime import date
from datetime import datetime
import time

In [ ]:
# Connect Google Drive to Colab
drive.mount('/content/gdrive')
# Create a variable to store the data path on your drive
path = './gdrive/My Drive/'

Mounted at /content/gdrive


In [ ]:
# Load Twitter API secrets from an external JSON file
#secrets = json.loads(open(path + 'secrets.json').read())
api_key = ''
api_secret_key = ''
access_token = ''
access_token_secret = ''
# Connect to Twitter API using the secrets
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
def save_json(file_name, file_content):
  with open(path + file_name, 'w', encoding='utf-8') as f:
    json.dump(file_content, f, ensure_ascii=False, indent=4)

In [ ]:
# Helper function to handle twitter API rate limit
def limit_handled(cursor, list_name):
  while True:
    try:
      yield cursor.next()
    # Catch Twitter API rate limit exception and wait for 15 minutes
    except tweepy.RateLimitError:
      print("\nData points in list = {}".format(len(list_name)))
      print('Hit Twitter API rate limit.')
      for i in range(3, 0, -1):
        print("Wait for {} mins.".format(i * 5))
        time.sleep(5 * 60)
    # Catch any other Twitter API exceptions
    except tweepy.error.TweepError:
      print('\nCaught TweepError exception' )

In [ ]:
# Helper function to get all tweets of a specified user
# NOTE:This method only allows access to the most recent 3200 tweets
# Source: https://gist.github.com/yanofsky/5436496
def get_all_tweets(screen_name):
  # initialize a list to hold all the Tweets
  alltweets = []
  # make initial request for most recent tweets 
  # (200 is the maximum allowed count)
  new_tweets = api.user_timeline(screen_name = screen_name,count=200)
  # save most recent tweets
  alltweets.extend(new_tweets)
  # save the id of the oldest tweet less one to avoid duplication
  oldest = alltweets[-1].id - 1
  # keep grabbing tweets until there are no tweets left
  while len(new_tweets) > 0:
    print("getting tweets before %s" % (oldest))
    # all subsequent requests use the max_id param to prevent
    # duplicates
    new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
    # save most recent tweets
    alltweets.extend(new_tweets)
    # update the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    print("...%s tweets downloaded so far" % (len(alltweets)))
    ### END OF WHILE LOOP ###
  # transform the tweepy tweets into a 2D array that will 
  # populate the csv
  outtweets = [[tweet.id_str, tweet.created_at, tweet.text, tweet.favorite_count,tweet.in_reply_to_screen_name, tweet.retweeted] for tweet in alltweets]
  # write the csv
  with open(path + '%s_tweets.csv' % screen_name, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["id","created_at","text","likes","in reply to","retweeted"])
    writer.writerows(outtweets)
  pass

In [ ]:
# Function to save follower objects in a JSON file.
def get_followers():
  
  # Create a list to store follower data
  followers_list = []
  # For-loop to iterate over tweepy cursors
  cursor = tweepy.Cursor(api.followers, count=200).pages()
  for i, page in enumerate(limit_handled(cursor, followers_list)):  
    print("\r"+"Loading"+ i % 5 *".", end='')
    
    # Add latest batch of follower data to the list
    followers_list += page
  
  # Extract the follower information
  followers_list = [x._json for x in followers_list]
  # Save the data in a JSON file
  save_json('followers_data.json', followers_list)

In [ ]:
# Function to save friend objects in a JSON file.
def get_friends():
  
  # Create a list to store friends data
  friends_list = []
  # For-loop to iterate over tweepy cursors
  cursor = tweepy.Cursor(api.friends, count=200).pages()
  for i, page in enumerate(limit_handled(cursor, friends_list)):  
    print("\r"+"Loading"+ i % 5 *".", end='')
    
    # Add latest batch of friend data to the list
    friends_list += page
  
  # Extract the friends information
  friends_list = [x._json for x in friends_list]
  # Save the data in a JSON file
  save_json('friends_data.json', friends_list)

In [ ]:
# Function to save daily follower and following counts in a JSON file
def todays_stats(dict_name):
  # Get my account information
  info = api.me()
  # Get follower and following counts
  followers_cnt = info.followers_count  
  following_cnt = info.friends_count
  # Get today's date
  today = date.today()
  d = today.strftime("%b %d, %Y")
  # Save today's stats only if they haven't been collected before
  if d not in dict_name:
    dict_name[d] = {"followers":followers_cnt, "following":following_cnt}
    save_json("follower_history.json", dict_name)
  else:
    print('Today\'s stats already exist')

In [ ]:
def get_tweet_hashtag(hashtag):
  # Open/Create a file to append data
  csvFile = open(path +'hashtag.csv', 'a')
  #Use csv Writer
  csvWriter = csv.writer(csvFile)

  for tweet in tweepy.Cursor(api.search,q="#"+hashtag,count=200,
                           lang="en",
                           since="2019-04-03").items():
    #print (tweet.created_at, tweet.text)
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

In [ ]:
def get_tweet_by_country(country,hashtag):

  places = api.geo_search(query=country, granularity="country")
  place_id = places[0].id
  for tweet in tweepy.Cursor(api.search ,
                            q="#"+hashtag + "&place:%s" % place_id,
                            lang="es").items():
  #for tweet in tweepy.Cursor(api.search,q=""+hashtag,
   #                         geocode="-33.45694,-70.64827,50000km",
    #                       ).items(4):
    print(f'{tweet.text} {tweet.id}')
  



In [ ]:
get_tweet_by_country("Chile","covid19")


Sexy science. 😏
#COVID19 mood😁 en Facultad de Farmacia UdeC https://t.co/fG5wXg6YtB 1313951422527016960
✅ La actualización sobre la disponibilidad de camas críticas registrada este miércoles en la macrored de salud de l… https://t.co/tRlAzUSo3U 1313945548391698432
Esaaa esss que pasa con el saludo vulcano!!! Jajajajjajj no hubiera esta mal uno para sordos mudos, ojo ahí!! Bueno… https://t.co/cyIpNBWqtm 1313886558739673090
📢 𝟮𝟱 nuevos casos de #Covid_19 fueron confirmados en la Región de Coquimbo, correspondientes a 10 de La Serena, 14… https://t.co/RiJ4JGvtKc 1313871821956055040
Escúchanos por Radio #JuanPabloII de #Illapel, mientras conversamos sobre #Covid19 #RegionDeCoquimbo #Cuarentena… https://t.co/Q02Nt92TxJ 1313867758493409288
Este virus como pandemia es una mentira #COVID19 lo vengo diciendo desde que me di cuenta que todo es una manipulac… https://t.co/pdeGRfLmcu 1313866669580746753
😷Continuando con las medidas preventivas frente al #Covid19, hoy nuevamente se están efectuando

In [ ]:
def get_tweet_by_id(id):
    tweet = api.get_status(id)
    print(tweet.text)

In [ ]:
get_tweet_hashtag("fake")

In [ ]:
get_all_tweets("FactCheckingcl");

getting tweets before 1135389206837178367
...292 tweets downloaded so far
getting tweets before 1113553967009685504
...292 tweets downloaded so far


In [ ]:
get_tweet_by_id(284329075902926848)

Did Miley Cyrus and Liam Hemsworth secretly get married? http://t.co/3vHeai3s
